In [ ]:
import os

# Set the working directory
working_directory = os.path.join(os.path.expanduser("~"), "Library", "CloudStorage", 
                              "OneDrive-UniversityofExeter", "Documents", "VISIONARY", 
                              "Durham Experiment", "processed_data_3")
os.chdir(working_directory)

# Verify the current working directory
print(f"Current working directory: {os.getcwd()}")

In [ ]:
from pathlib import Path

# Get the .mp4 files in the folder
mp4_files = list(Path(working_directory).glob("*.mp4"))
# Print the .mp4 files without showing the parent directories
print([file.name for file in mp4_files])

In [ ]:
# Filter the files for Camera_1 and Camera_2
camera_1_files = [file for file in mp4_files if file.name.startswith("Camera_1_")]
camera_2_files = [file for file in mp4_files if file.name.startswith("Camera_2_")]

# Sort the files by date extracted from the filename
camera_1_files_sorted = sorted(camera_1_files, key=lambda x: x.stem.split('_')[-1])
camera_2_files_sorted = sorted(camera_2_files, key=lambda x: x.stem.split('_')[-1])

print("Camera 1 files sorted by date:", [file.name for file in camera_1_files_sorted])
print("Camera 2 files sorted by date:", [file.name for file in camera_2_files_sorted])

del mp4_files, camera_1_files, camera_2_files

In [ ]:
import cv2
import json
import numpy as np

def draw_door_area(video_path, save_path):
    # Read the first frame of the video
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    # Create a copy of the original frame
    original_frame = frame.copy()
    drawing_frame = frame.copy()
    
    # Store the coordinates of the rectangle
    door_coords = []
    
    def mouse_callback(event, x, y, flags, param):
        nonlocal drawing_frame
        
        if event == cv2.EVENT_LBUTTONDOWN:
            # Reset if we already have 2 points
            if len(door_coords) == 2:
                door_coords.clear()
                drawing_frame = original_frame.copy()
            
            door_coords.append((x, y))
            # Draw point for visual feedback
            cv2.circle(drawing_frame, (x, y), 3, (0, 255, 0), -1)
            
            if len(door_coords) == 2:
                # Draw rectangle using the two points
                cv2.rectangle(drawing_frame, door_coords[0], door_coords[1], (0, 255, 0), 2)
                # Add instruction text
                cv2.putText(drawing_frame, "Press 'r' to redraw, Enter to confirm", 
                          (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow('Frame', drawing_frame)

    # Create window and set mouse callback
    cv2.imshow('Frame', drawing_frame)
    cv2.setMouseCallback('Frame', mouse_callback)
    
    # Instructions
    print("Instructions:")
    print("1. Click two points to draw a rectangle")
    print("2. Press 'r' to redraw if needed")
    print("3. Press Enter to confirm and save")
    print("4. Press 'q' to quit without saving")
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # Press 'r' to reset
        if key == ord('r'):
            door_coords.clear()
            drawing_frame = original_frame.copy()
            cv2.imshow('Frame', drawing_frame)
        
        # Press Enter to confirm
        elif key == 13:  # Enter key
            if len(door_coords) == 2:
                # Ensure coordinates are in top-left, bottom-right order
                x1, y1 = min(door_coords[0][0], door_coords[1][0]), min(door_coords[0][1], door_coords[1][1])
                x2, y2 = max(door_coords[0][0], door_coords[1][0]), max(door_coords[0][1], door_coords[1][1])
                
                coordinates = {
                    'top_left': {'x': x1, 'y': y1},
                    'bottom_right': {'x': x2, 'y': y2}
                }
                with open(save_path, 'w') as f:
                    json.dump(coordinates, f)
                print(f"\nDoor coordinates saved to {save_path}:")
                print(f"Top-left: ({x1}, {y1})")
                print(f"Bottom-right: ({x2}, {y2})")
                break
            else:
                print("Please complete the rectangle before confirming")
        
        # Press 'q' to quit
        elif key == ord('q'):
            print("\nQuitting without saving")
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Usage example
video_path = camera_1_files_sorted[0]
draw_door_area(video_path, save_path="door_coordinates_camera1.json")

#video_path = camera_2_files_sorted[0]
#draw_door_area(video_path, save_path="door_coordinates_camera2.json")

In [ ]:
Door coordinates saved to door_coordinates_camera1.json:
        Top-left: (1033, 9)
        Bottom-right: (1673, 559)


Top-left: (459, 2)
Bottom-right: (739, 469)

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from scipy.spatial.distance import cosine
import urllib.request
import os
from collections import deque


class PersonTracker:
    def __init__(self):
        self.model = YOLO('yolov8x.pt')

        # Download and load face cascade file
        cascade_file = 'haarcascade_frontalface_default.xml'
        if not os.path.exists(cascade_file):
            url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
            urllib.request.urlretrieve(url, cascade_file)

        self.face_cascade = cv2.CascadeClassifier(cascade_file)

        self.unique_persons = []
        self.position_history = []
        self.track_length = 30

        # Tracking parameters
        self.similarity_threshold = 0.7
        self.position_threshold = 150

        # Door zone parameters
        self.door_zone = None  # Will be set by user

    def process_video(self, video_path):
        # Initialize VideoCapture
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error: Unable to open video file: {video_path}")
            return

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video or failed to read frame.")
                break

            frame_count += 1
            print(f"Processing frame {frame_count}...")

            # Preprocess frame
            frame = cv2.resize(frame, (640, 384))

            # YOLO Inference
            try:
                results = self.model(frame)
            except Exception as e:
                print(f"Error during inference: {e}")
                break

            detections = results[0].boxes.data.cpu().numpy()
            print(f"Detections: {len(detections)}")

            # Draw detections
            for det in detections:
                x1, y1, x2, y2, conf, cls = det
                cv2.rectangle(
                    frame,
                    (int(x1), int(y1)),
                    (int(x2), int(y2)),
                    (0, 255, 0),
                    2
                )
                cv2.putText(
                    frame,
                    f"{conf:.2f}",
                    (int(x1), int(y1 - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    2
                )

            # Display frame
            cv2.imshow("Frame", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

tracker = PersonTracker()
unique_count = tracker.process_video(video_path)
print(f"Number of unique individuals detected: {unique_count}")

# Backup

In [ ]:
class PersonTracker:
    def __init__(self, video_path, output_dir="tracked_persons"):
        # Initialize YOLO model
        self.detector = YOLO("yolo11x.pt")
        
        # Initialize ReID model
        self.reid_model = torchreid.models.build_model(
            name='osnet_x1_0',
            num_classes=1000,
            pretrained=True
        )
        self.reid_model = self.reid_model.cuda() if torch.cuda.is_available() else self.reid_model
        self.reid_model.eval()
        
        # Initialize video capture
        self.cap = cv2.VideoCapture(video_path)
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        
        # Create output directory
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
        # Initialize tracking variables
        self.active_tracks = {}  # Currently active tracks
        self.person_features = {}  # Historical features for each ID
        self.person_timestamps = {}  # Timestamp information
        self.next_id = 0
        
        # Tracking parameters
        self.similarity_threshold = 0.7
        self.max_disappeared = self.fps * 2  # Max frames to keep track without detection
        self.min_detection_confidence = 0.5
        self.feature_weight = 0.4   # Weight for ReID features in matching
        self.position_weight = 0.3  # Weight for absolute position (IoU)
        self.motion_weight = 0.3    # Weight for relative motion prediction
        
    def extract_features(self, person_crop):
        """Extract ReID features from person crop"""
        try:
            # Preprocess image for ReID
            img = cv2.resize(person_crop, (128, 256))
            img = torch.from_numpy(img).float()
            img = img.permute(2, 0, 1).unsqueeze(0)
            if torch.cuda.is_available():
                img = img.cuda()
                
            # Extract features
            with torch.no_grad():
                features = self.reid_model(img)
            return features.cpu().numpy()
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None
            
    def calculate_box_center(self, box):
        """Calculate center point of a bounding box"""
        return [(box[0] + box[2]) / 2, (box[1] + box[3]) / 2]

    def calculate_velocity(self, current_box, previous_box):
        """Calculate velocity vector between two boxes"""
        current_center = self.calculate_box_center(current_box)
        previous_center = self.calculate_box_center(previous_box)
        return [current_center[0] - previous_center[0], 
                current_center[1] - previous_center[1]]

    def predict_next_position(self, box, velocity):
        """Predict next position based on current position and velocity"""
        center = self.calculate_box_center(box)
        predicted_center = [center[0] + velocity[0], center[1] + velocity[1]]
        width = box[2] - box[0]
        height = box[3] - box[1]
        return [predicted_center[0] - width/2, predicted_center[1] - height/2,
                predicted_center[0] + width/2, predicted_center[1] + height/2]

    def calculate_motion_similarity(self, current_boxes, tracked_boxes, tracked_velocities):
        """Calculate motion-based similarity"""
        n_detections = len(current_boxes)
        n_tracks = len(tracked_boxes)
        motion_sim = np.zeros((n_detections, n_tracks))
        
        for i, current_box in enumerate(current_boxes):
            current_center = self.calculate_box_center(current_box)
            for j, (tracked_box, velocity) in enumerate(zip(tracked_boxes, tracked_velocities)):
                # Predict where the tracked box should be
                predicted_box = self.predict_next_position(tracked_box, velocity)
                predicted_center = self.calculate_box_center(predicted_box)
                
                # Calculate distance between prediction and actual position
                distance = np.sqrt(
                    (current_center[0] - predicted_center[0])**2 +
                    (current_center[1] - predicted_center[1])**2
                )
                # Convert distance to similarity (closer = more similar)
                motion_sim[i, j] = np.exp(-distance / 100.0)  # 100 is a scaling factor
                
        return motion_sim

    def detect_occlusion(self, box1, box2):
        """
        Detect if box1 is occluded by box2.
        Returns: 
            - is_occluded (bool): True if box1 is occluded by box2
            - occlusion_score (float): Degree of occlusion (0 to 1)
        """
        # Calculate IoU
        iou = self.calculate_iou(box1, box2)
        
        # Calculate centers and areas
        center1 = self.calculate_box_center(box1)
        center2 = self.calculate_box_center(box2)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        # Calculate vertical position (y-coordinate)
        y1 = box1[3]  # bottom of box1
        y2 = box2[3]  # bottom of box2
        
        # Factors that suggest box1 is behind box2:
        # 1. Significant overlap
        overlap_factor = 1.0 if iou > 0.3 else 0.0
        
        # 2. Box2 is closer to camera (generally larger and lower in frame)
        size_factor = 1.0 if area2 > area1 else 0.0
        position_factor = 1.0 if y2 > y1 else 0.0
        
        # 3. Box1 is partially contained within box2
        contained_horizontally = (
            (box1[0] > box2[0] and box1[0] < box2[2]) or
            (box1[2] > box2[0] and box1[2] < box2[2])
        )
        contained_vertically = (
            (box1[1] > box2[1] and box1[1] < box2[3]) or
            (box1[3] > box2[1] and box1[3] < box2[3])
        )
        containment_factor = 1.0 if (contained_horizontally and contained_vertically) else 0.0
        
        # Calculate occlusion score (weighted combination of factors)
        occlusion_score = (
            0.4 * overlap_factor +
            0.2 * size_factor +
            0.2 * position_factor +
            0.2 * containment_factor
        )
        
        # Determine if occluded based on score threshold
        is_occluded = occlusion_score > 0.5
        
        return is_occluded, occlusion_score

    def calculate_similarity_matrix(self, current_features, current_boxes, tracked_features, tracked_boxes):
        """Calculate similarity matrix combining appearance, position, and motion"""
        n_detections = len(current_features)
        n_tracks = len(tracked_features)
        
        if n_detections == 0 or n_tracks == 0:
            return np.array([])
            
        # Calculate appearance similarity
        appearance_sim = 1 - distance.cdist(
            np.array([f.flatten() for f in current_features]), 
            np.array([f.flatten() for f in tracked_features]), 
            metric='cosine'
        )
        
        # Calculate position similarity using IoU
        position_sim = np.zeros((n_detections, n_tracks))
        for i, box1 in enumerate(current_boxes):
            for j, box2 in enumerate(tracked_boxes):
                position_sim[i, j] = self.calculate_iou(box1, box2)
        
        # Calculate velocities for tracked objects
        tracked_velocities = []
        for track_id in list(self.active_tracks.keys())[:n_tracks]:
            if 'previous_box' in self.active_tracks[track_id]:
                velocity = self.calculate_velocity(
                    self.active_tracks[track_id]['box'],
                    self.active_tracks[track_id]['previous_box']
                )
            else:
                velocity = [0, 0]  # No velocity for new tracks
            tracked_velocities.append(velocity)
        
        # Calculate motion similarity
        motion_sim = self.calculate_motion_similarity(current_boxes, tracked_boxes, tracked_velocities)
        
        # Combine all similarities
        similarity_matrix = (
            self.feature_weight * appearance_sim + 
            self.position_weight * position_sim +
            self.motion_weight * motion_sim
        )
        
        return similarity_matrix
    
    @staticmethod
    def calculate_iou(box1, box2):
        """Calculate IoU between two boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / (union + 1e-6)
    
    def update_feature_history(self, track_id, features):
        """Maintain rolling window of recent features"""
        self.appearance_history[track_id].append(features)
        if len(self.appearance_history[track_id]) > self.max_history_length:
            self.appearance_history[track_id].pop(0)
            
        # Update feature representation using exponential moving average
        if track_id in self.person_features:
            alpha = 0.7  # Weight for historical features
            current_features = self.person_features[track_id]
            updated_features = alpha * current_features + (1 - alpha) * features
            self.person_features[track_id] = updated_features
        else:
            self.person_features[track_id] = features

    def recover_lost_tracklet(self, features, current_box, frame_time):
        """Attempt to recover lost tracks"""
        best_match_id = None
        best_match_score = 0
        
        # Check recently lost tracks
        lost_tracks_to_remove = []
        for lost_id, lost_info in self.lost_tracks.items():
            # Skip if lost track is too old
            if frame_time - lost_info['last_seen'] > self.max_lost_age:
                lost_tracks_to_remove.append(lost_id)
                continue
                
            # Calculate appearance similarity
            lost_features = lost_info['features']
            appearance_sim = 1 - distance.cosine(features.flatten(), lost_features.flatten())
            
            # Calculate position similarity based on predicted movement
            predicted_box = self.predict_next_position(
                lost_info['box'],
                lost_info['velocity']
            )
            position_sim = self.calculate_iou(current_box, predicted_box)
            
            # Combine similarities
            match_score = (
                self.feature_weight * appearance_sim +
                self.position_weight * position_sim
            )
            
            # Check temporal consistency
            if match_score > 0.6 and match_score > best_match_score:
                best_match_score = match_score
                best_match_id = lost_id
        
        # Clean up old lost tracks
        for lost_id in lost_tracks_to_remove:
            del self.lost_tracks[lost_id]
            
        return best_match_id if best_match_score > 0.6 else None

    def update_tracks(self, frame, detections, frame_time):
        """Update tracks with new detections"""
        current_boxes = []
        current_features = []
        
        # Process new detections
        for box, conf in detections:
            if conf < self.min_detection_confidence:
                continue
                
            x1, y1, x2, y2 = map(int, box)
            person_crop = frame[y1:y2, x1:x2]
            if person_crop.size == 0:
                continue
                
            features = self.extract_features(person_crop)
            if features is not None:
                current_boxes.append([x1, y1, x2, y2])
                current_features.append(features)
        
        # Get tracked boxes and features
        tracked_boxes = []
        tracked_features = []
        tracked_ids = []
        
        for track_id, track_info in self.active_tracks.items():
            tracked_boxes.append(track_info['box'])
            tracked_features.append(track_info['features'])
            tracked_ids.append(track_id)
        
        # Calculate similarity matrix
        similarity_matrix = self.calculate_similarity_matrix(
            current_features, current_boxes,
            tracked_features, tracked_boxes
        )
        
        # Perform matching
        matched_indices = []
        if similarity_matrix.size > 0:
            row_ind, col_ind = linear_sum_assignment(-similarity_matrix)
            matched_indices = list(zip(row_ind, col_ind))
        
        # Process matches
        unmatched_detections = []
        matched_track_ids = set()
        
        for detection_idx, track_idx in matched_indices:
            similarity = similarity_matrix[detection_idx, track_idx]
            if similarity >= self.similarity_threshold:
                track_id = tracked_ids[track_idx]
                matched_track_ids.add(track_id)
                
                # Update track
                self.active_tracks[track_id].update({
                    'previous_box': self.active_tracks[track_id]['box'],
                    'box': current_boxes[detection_idx],
                    'features': current_features[detection_idx],
                    'last_seen': frame_time,
                    'disappeared': 0
                })
                
                # Update timestamps
                self.person_timestamps[track_id]['last_appearance'] = frame_time
                
                # Save person image
                self.save_person_image(track_id, 
                    frame[current_boxes[detection_idx][1]:current_boxes[detection_idx][3],
                          current_boxes[detection_idx][0]:current_boxes[detection_idx][2]])
            else:
                unmatched_detections.append(detection_idx)
        
        # Add unmatched detections as new tracks
        for detection_idx in range(len(current_features)):
            if not any(detection_idx == m[0] for m in matched_indices):
                new_id = self.next_id
                self.next_id += 1
                
                self.active_tracks[new_id] = {
                    'state': TrackingState.TENTATIVE,
                    'occlusion_counter': 0,
                    'box': current_boxes[detection_idx],
                    'features': current_features[detection_idx],
                    'last_seen': frame_time,
                    'disappeared': 0,
                    'velocity': [0, 0]  # Initialize velocity for new tracks
                }
                
                self.person_features[new_id] = [current_features[detection_idx]]
                self.person_timestamps[new_id] = {
                    'first_appearance': frame_time,
                    'last_appearance': frame_time
                }
                
                # Save person image
                self.save_person_image(new_id, 
                    frame[current_boxes[detection_idx][1]:current_boxes[detection_idx][3],
                          current_boxes[detection_idx][0]:current_boxes[detection_idx][2]])
        
        # Update disappeared tracks
        current_time = frame_time
        tracks_to_remove = []
        
        for track_id in self.active_tracks:
            if track_id not in matched_track_ids:
                self.active_tracks[track_id]['disappeared'] += 1
                if self.active_tracks[track_id]['disappeared'] > self.max_disappeared:
                    tracks_to_remove.append(track_id)
        
        # Remove old tracks
        for track_id in tracks_to_remove:
            del self.active_tracks[track_id]
    
    def save_person_image(self, person_id, frame):
        """Save person image to output directory"""
        person_dir = os.path.join(self.output_dir, f"person_{person_id}")
        os.makedirs(person_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        cv2.imwrite(os.path.join(person_dir, f"{timestamp}.jpg"), frame)
    
    def process_video(self):
        frame_count = 0
        
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
                
            frame_time = frame_count / self.fps
            frame_count += 1
            
            # Detect persons using YOLO
            results = self.detector(frame, classes=[0])  # class 0 is person
            
            # Process detections
            detections = []
            for result in results:
                boxes = result.boxes.cpu().numpy()
                for box in boxes:
                    detections.append((box.xyxy[0], box.conf[0]))
            
            # Update tracking
            self.update_tracks(frame, detections, frame_time)
            
            # Visualize results
            for track_id, track_info in self.active_tracks.items():
                box = track_info['box']
                cv2.rectangle(frame, (int(box[0]), int(box[1])), 
                            (int(box[2]), int(box[3])), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", 
                          (int(box[0]), int(box[1])-10),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Display frame
            cv2.imshow('Tracking', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        self.cap.release()
        cv2.destroyAllWindows()
        
        return self.generate_report()
    
    def generate_report(self):
        """Generate tracking report"""
        report = {
            'total_unique_persons': self.next_id,
            'person_details': {}
        }
        
        for person_id in self.person_timestamps.keys():
            report['person_details'][person_id] = {
                'first_appearance': self.person_timestamps[person_id]['first_appearance'],
                'last_appearance': self.person_timestamps[person_id]['last_appearance'],
                'duration': self.person_timestamps[person_id]['last_appearance'] - 
                          self.person_timestamps[person_id]['first_appearance'],
                'image_path': os.path.join(self.output_dir, f"person_{person_id}")
            }
            
        return report
